# NBA Data :: Fixing

In [16]:
from nba_api.stats.endpoints import LeagueGameFinder, BoxScoreTraditionalV2
import pandas as pd
import time
from datetime import datetime
from tqdm.auto import tqdm
import logging
import random

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('nba_data_collection.log'),
             logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

def log_message(message):
    """Log message to both file and tqdm"""
    tqdm.write(message)
    logger.info(message)

In [17]:
import pandas as pd
from nba_api.stats.endpoints import LeagueGameFinder, PlayByPlay
df = PlayByPlay('0040400407').get_data_frames()[0]
df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0040400407,0,12,0,1,9:36 PM,12:00,None,Start of 1st Period (9:36 PM EST),None,None,None
1,0040400407,1,10,0,1,9:36 PM,12:00,Jump Ball Mohammed vs. B. Wallace: Tip to Parker,None,None,None,None
2,0040400407,2,1,66,1,9:37 PM,11:41,Duncan 9' Jump Bank Shot (2 PTS),None,None,0 - 2,2
3,0040400407,3,6,1,1,9:38 PM,11:21,Ginobili P.FOUL (P1.T1),None,None,None,None
4,0040400407,4,1,1,1,9:38 PM,11:11,None,None,R. Wallace 12' Jump Shot (2 PTS),2 - 2,TIE
...,...,...,...,...,...,...,...,...,...,...,...,...
405,0040400407,431,3,11,4,12:18 AM,0:07,Ginobili Free Throw 1 of 2 (22 PTS),None,None,74 - 80,6
406,0040400407,432,3,12,4,12:19 AM,0:07,Ginobili Free Throw 2 of 2 (23 PTS),None,None,74 - 81,7
407,0040400407,433,2,1,4,12:19 AM,0:00,None,None,MISS Prince 26' 3PT Jump Shot,None,None
408,0040400407,434,4,0,4,12:19 AM,0:00,None,None,Pistons Rebound,None,None


In [18]:
def get_season_games(season):
    """Get games for a single season with error handling"""
    game_finder = LeagueGameFinder(
        season_nullable=season,
        league_id_nullable='00',
        timeout=60
    )
    
    # Get the raw response first
    response_frames = game_finder.get_data_frames()
    
    # Debug logging
    log_message(f"Response for season {season}: got {len(response_frames)} DataFrames")
    if not response_frames:
        raise ValueError(f"Empty response for season {season}")
    
    games = response_frames[0]
    if len(games) == 0:
        raise ValueError(f"No games found for season {season}")
        
    log_message(f"Retrieved {len(games)} game entries for season {season}")
    return games

def compile_game_ids(start_year, end_year):
    game_ids = []
    seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(start_year, end_year)]

    for season in seasons:
        game_ids.extend(get_season_games(season)['GAME_ID'].unique())
        time.sleep(1)
    return game_ids

In [19]:
dfs = list()
l = compile_game_ids(2023,2024)
for idx in l:
    df = PlayByPlay('0040400407').get_data_frames()[0]
    dfs.append(df)


2025-02-21 21:44:50,853 - INFO - Response for season 2023-24: got 1 DataFrames
2025-02-21 21:44:50,856 - INFO - Retrieved 2795 game entries for season 2023-24


Response for season 2023-24: got 1 DataFrames
Retrieved 2795 game entries for season 2023-24


KeyboardInterrupt: 